In [1]:
import time
import logging
import lithops
from radiointerferometry.utils import setup_logging, get_executor_id_lithops
from radiointerferometry.steps.imaging import ImagingStep
from radiointerferometry.steps.pipelinestep import DP3Step
from radiointerferometry.datasource import InputS3, OutputS3
from radiointerferometry.partitioning import StaticPartitioner


# Logger setup
LOG_LEVEL = logging.INFO
logger = setup_logging(LOG_LEVEL)
partitioner = StaticPartitioner(log_level=LOG_LEVEL)

BUCKET = "os-10gb"
RACK_BUCKET = "os-10gb"

def prepend_hash_to_key(key: str) -> str:
    #print(f"Executor ID: {get_executor_id_lithops()}")
    return f"440531/{key}"


fexec = lithops.FunctionExecutor(
    log_level=LOG_LEVEL, runtime_memory=2048, runtime_cpu=4
)

# Input ms's are stored here
inputs = InputS3(bucket=BUCKET, key="CYGLOOP2024/20240312_081800_20240312_084100_CYGLOOP_CYGA/")



2024-06-26 15:09:05 [INFO] Started StaticPartitioner
2024-06-26 15:09:05,008 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:09:06,945 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:06,979 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default


Initialized InputS3 with bucket: os-10gb, key: CYGLOOP2024/20240312_081800_20240312_084100_CYGLOOP_CYGA/, file_ext: None, dynamic: False, base_local_path: /tmp


In [2]:
# Workflow is described like this:
# CALIBRATOR:  [FLAG&REBIN] -> [CALIBRATION] -> (caltables HDF5 files)
# TARGET: [FLAG&REBIN] -> [CALIBRATION (APPLYCAL ONLY)] -> [IMAGING]

# Rebinning parameters with hash included in the key as a root directory, notice how we use the result from the partitioning step

# CALIBRATOR REBINNING PARAMS
CAL_rebinning_params = {
    "msin": inputs,
    "steps": "[aoflag, avg, count]",
    "aoflag.type": "aoflagger",
    "aoflag.strategy": InputS3(
        bucket=BUCKET,
        key="parameters/rebinning/STEP1-NenuFAR64C1S.lua",
    ),
    "avg.type": "averager",
    "avg.freqstep": 5,
    "avg.timestep": 2,
    "msout": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/rebinning_out/ms"),
        file_ext="ms",
    ),
    "numthreads": 4,
    "log_output": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/rebinning_out/logs"),
        file_ext="log",
    ),
}


 # CALIBRATOR REBINNING
start_time = time.time()
finished_job = DP3Step(
    parameters=CAL_rebinning_params,
    log_level=LOG_LEVEL
).run(func_limit=1)

end_time = time.time()
logger.info(f"CAL Rebinning completed in {end_time - start_time} seconds.")


2024-06-26 15:09:07,002 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:09:07,015 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:07,046 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default
2024-06-26 15:09:07,059 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:07 [INFO] keys : ['CYGLOOP2024/20240312_081800_20240312_084100_CYGLOOP_CYGA/SB220.MS.zip']
2024-06-26 15:09:07,107 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:07 [INFO] New output path: 440531/CAL/rebinning_out/ms/SB220.ms
2024-06-26 15:09:07 [INFO] New output path: 440531/CAL/rebinning_out/logs/SB220.log
2024-06-26 15:09:07 [INFO] Function params: [[{'msin': /tmp/os-10gb/CYGLOOP2024/20240312_081800_20240312_084100_CYGLOOP_CYGA/SB220.MS.zip, 'steps': '[aoflag, avg, count]', 'aoflag.type': 'aoflagger', 'aoflag.strategy': /tmp/os-10gb/parameters/rebi

Initialized InputS3 with bucket: os-10gb, key: parameters/rebinning/STEP1-NenuFAR64C1S.lua, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/ms, file_ext: ms, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/logs, file_ext: log, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: CYGLOOP2024/20240312_081800_20240312_084100_CYGLOOP_CYGA/SB220.MS.zip, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/ms/SB220.ms, file_ext: None, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/logs/SB220.log, file_ext: None, file_name: None, remote_key_ow: None, base_local_path: /tmp


2024-06-26 15:09:08,333 [INFO] invokers.py:174 -- ExecutorID 37a208-1 | JobID M000 - Starting function invocation: _execute_step() - Total: 1 activations
2024-06-26 15:09:08,443 [INFO] invokers.py:213 -- ExecutorID 37a208-1 | JobID M000 - View execution logs at /tmp/lithops-ayman/logs/37a208-1-M000.log
2024-06-26 15:09:08,446 [INFO] executors.py:491 -- ExecutorID 37a208-1 - Getting results from 1 function activations
2024-06-26 15:09:08,446 [INFO] wait.py:101 -- ExecutorID 37a208-1 - Waiting for 1 function activations to complete
/usr/local/lib/python3.10/dist-packages/tqdm-4.66.1-py3.10.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-26 15:09:25,523 [INFO] executors.py:615 -- ExecutorID 37a208-1 - Cleaning temporary data
2024-06-26 15:09:25 [INFO] CAL Rebinning completed in 18.52946138381958 seconds.


In [3]:
CAL_calibration_params = {
    "msin": InputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/rebinning_out/ms"),
    ),
    "msin.datacolumn": "DATA",
    "msout": ".",
    "steps": "[cal]",
    "cal.type": "gaincal",
    "cal.caltype": "diagonal",
    "cal.sourcedb": InputS3(
        bucket=BUCKET,
        key="parameters/calibration/CAL.sourcedb",
    ),
    "cal.parmdb": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/calibration_out/h5"),
        file_ext="h5",
    ),
    "cal.solint": 0, # means 1 solution for all time steps  
    "cal.nchan": 1, # means 1 solution per channel
    "cal.maxiter": 50,
    "cal.uvlambdamin": 5,
    "cal.smoothnessconstraint": 2e6,
    "numthreads": 4,
    "log_output": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/calibration_out/logs"),
        file_ext="log",
    ),
}

# CALIBRATOR CALIBRATION
start_time = time.time()

finished_job = DP3Step(
    parameters=CAL_calibration_params,
    log_level=LOG_LEVEL
).run(func_limit=1)

end_time = time.time()
logger.info(f"CAL Calibration completed in {end_time - start_time} seconds.")

2024-06-26 15:09:25,554 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:09:25,569 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000


2024-06-26 15:09:25,604 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default
2024-06-26 15:09:25,617 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:25 [INFO] keys : ['440531/CAL/rebinning_out/ms/SB220.ms.zip']
2024-06-26 15:09:25,642 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:25 [INFO] New output path: 440531/CAL/calibration_out/h5/SB220.h5
2024-06-26 15:09:25 [INFO] New output path: 440531/CAL/calibration_out/logs/SB220.log
2024-06-26 15:09:25 [INFO] Function params: [[{'msin': /tmp/os-10gb/440531/CAL/rebinning_out/ms/SB220.ms.zip, 'msin.datacolumn': 'DATA', 'msout': '.', 'steps': '[cal]', 'cal.type': 'gaincal', 'cal.caltype': 'diagonal', 'cal.sourcedb': /tmp/os-10gb/parameters/calibration/CAL.sourcedb, 'cal.parmdb': OutputS3(bucket=os-10gb, key=440531/CAL/calibration_out/h5/SB220.h5, file_ext=None, file_name=None, remote_key_ow=None, base_local_path=/tmp), 'cal.solin

Initialized InputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/ms, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: parameters/calibration/CAL.sourcedb, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/h5, file_ext: h5, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/logs, file_ext: log, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: 440531/CAL/rebinning_out/ms/SB220.ms.zip, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/h5/SB220.h5, file_ext: None, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/logs/SB220.log, file_ext: None, file_name

2024-06-26 15:09:26,749 [INFO] invokers.py:174 -- ExecutorID 37a208-2 | JobID M000 - Starting function invocation: _execute_step() - Total: 1 activations
2024-06-26 15:09:26,865 [INFO] invokers.py:213 -- ExecutorID 37a208-2 | JobID M000 - View execution logs at /tmp/lithops-ayman/logs/37a208-2-M000.log
2024-06-26 15:09:26,870 [INFO] executors.py:491 -- ExecutorID 37a208-2 - Getting results from 1 function activations
2024-06-26 15:09:26,871 [INFO] wait.py:101 -- ExecutorID 37a208-2 - Waiting for 1 function activations to complete
2024-06-26 15:09:35,898 [INFO] executors.py:615 -- ExecutorID 37a208-2 - Cleaning temporary data
2024-06-26 15:09:35 [INFO] CAL Calibration completed in 10.34995722770691 seconds.


In [4]:
inputs_tar = InputS3(bucket=BUCKET, key="CYGLOOP2024/20240312_084100_20240312_100000_CYGLOOP_TARGET/")



# TARGET REBINNING PARAMS
TARGET_rebinning_params = {
    "msin": inputs_tar,
    "steps": "[aoflag, avg, count]",
    "aoflag.type": "aoflagger",
    "aoflag.strategy": InputS3(
        bucket=BUCKET,
        key="parameters/rebinning/STEP1-NenuFAR64C1S.lua",
    ),
    "avg.type": "averager",
    "avg.freqstep": 5, # averaging 5 channels
    "avg.timestep": 2, # averaging 2 times samples
    "msout": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("TAR/rebinning_out/ms"),
        file_ext="ms",
    ),
    "numthreads": 4,
    "log_output": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("TAR/rebinning_out/logs"),
        file_ext="log",
    ),
}

# TARGET REBINNING
start_time = time.time()
finished_job = DP3Step(
    parameters=TARGET_rebinning_params,
    log_level=LOG_LEVEL
).run(func_limit=1)

end_time = time.time()
logger.info(f"TARGET Rebinning completed in {end_time - start_time} seconds.")

2024-06-26 15:09:35,924 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:09:35,938 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:35,969 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default
2024-06-26 15:09:35,983 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:36 [INFO] keys : ['CYGLOOP2024/20240312_084100_20240312_100000_CYGLOOP_TARGET/SB220.MS.zip']
2024-06-26 15:09:36,017 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:09:36 [INFO] New output path: 440531/TAR/rebinning_out/ms/SB220.ms
2024-06-26 15:09:36 [INFO] New output path: 440531/TAR/rebinning_out/logs/SB220.log
2024-06-26 15:09:36 [INFO] Function params: [[{'msin': /tmp/os-10gb/CYGLOOP2024/20240312_084100_20240312_100000_CYGLOOP_TARGET/SB220.MS.zip, 'steps': '[aoflag, avg, count]', 'aoflag.type': 'aoflagger', 'aoflag.strategy': /tmp/os-10gb/parameters/

Initialized InputS3 with bucket: os-10gb, key: CYGLOOP2024/20240312_084100_20240312_100000_CYGLOOP_TARGET/, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: parameters/rebinning/STEP1-NenuFAR64C1S.lua, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms, file_ext: ms, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/logs, file_ext: log, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: CYGLOOP2024/20240312_084100_20240312_100000_CYGLOOP_TARGET/SB220.MS.zip, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms/SB220.ms, file_ext: None, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440

2024-06-26 15:09:37,052 [INFO] invokers.py:174 -- ExecutorID 37a208-3 | JobID M000 - Starting function invocation: _execute_step() - Total: 1 activations
2024-06-26 15:09:37,165 [INFO] invokers.py:213 -- ExecutorID 37a208-3 | JobID M000 - View execution logs at /tmp/lithops-ayman/logs/37a208-3-M000.log
2024-06-26 15:09:37,168 [INFO] executors.py:491 -- ExecutorID 37a208-3 - Getting results from 1 function activations
2024-06-26 15:09:37,170 [INFO] wait.py:101 -- ExecutorID 37a208-3 - Waiting for 1 function activations to complete
2024-06-26 15:10:18,244 [INFO] executors.py:615 -- ExecutorID 37a208-3 - Cleaning temporary data
2024-06-26 15:10:18 [INFO] TARGET Rebinning completed in 42.32728862762451 seconds.


In [5]:
TARGET_apply_calibration = {
    "msin": InputS3(bucket=BUCKET, key=prepend_hash_to_key("TAR/rebinning_out/ms")),
    "msin.datacolumn": "DATA",
    "msout": OutputS3(
       bucket=BUCKET,
       key=prepend_hash_to_key("TAR/rebinning_out/ms"),
       file_ext="ms",
       remote_key_ow=prepend_hash_to_key("TAR/applycal_out/ms"),
    ),
    "msout.datacolumn": "CORRECTED_DATA",
    "steps": "[apply]",
    "apply.type": "applycal",
    "apply.steps": "[apply_amp,apply_phase]",
    "apply.apply_amp.correction": "amplitude000",
    "apply.apply_phase.correction": "phase000",
    "apply.direction": "[Main]",
    "apply.parmdb": InputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("CAL/calibration_out/h5"),
        dynamic=True,
        file_ext="h5",
    ),
    "log_output": OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("TAR/applycal_out/logs"),
        file_ext="log",
    ),
}


# TARGET CALIBRATION (APPLY)
start_time = time.time()
finished_job = DP3Step(
    parameters=TARGET_apply_calibration,
    log_level=LOG_LEVEL
).run(func_limit=1)

end_time = time.time()
logger.info(f"target Calibration completed in {end_time - start_time} seconds.")

2024-06-26 15:10:18,276 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:10:18,290 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:10:18,320 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default
2024-06-26 15:10:18,334 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:10:18 [INFO] keys : ['440531/TAR/rebinning_out/ms/SB220.ms.zip']
2024-06-26 15:10:18,359 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:10:18 [INFO] New output path: 440531/TAR/rebinning_out/ms/SB220.ms
2024-06-26 15:10:18 [INFO] New output path: 440531/TAR/applycal_out/logs/SB220.log
2024-06-26 15:10:18 [INFO] Function params: [[{'msin': /tmp/os-10gb/440531/TAR/rebinning_out/ms/SB220.ms.zip, 'msin.datacolumn': 'DATA', 'msout': OutputS3(bucket=os-10gb, key=440531/TAR/rebinning_out/ms/SB220.ms, file_ext=None, file_name=None, remote_key_ow=440531/TAR/applycal

Initialized InputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms, file_ext: ms, file_name: None, remote_key_ow: 440531/TAR/applycal_out/ms, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/h5, file_ext: h5, dynamic: True, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/applycal_out/logs, file_ext: log, file_name: None, remote_key_ow: None, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms/SB220.ms.zip, file_ext: None, dynamic: False, base_local_path: /tmp
Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/rebinning_out/ms/SB220.ms, file_ext: None, file_name: None, remote_key_ow: 440531/TAR/applycal_out/ms, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: 440531/CAL/calibration_out/h5/SB220.h5, f

2024-06-26 15:10:19,271 [INFO] invokers.py:174 -- ExecutorID 37a208-4 | JobID M000 - Starting function invocation: _execute_step() - Total: 1 activations
2024-06-26 15:10:19,373 [INFO] invokers.py:213 -- ExecutorID 37a208-4 | JobID M000 - View execution logs at /tmp/lithops-ayman/logs/37a208-4-M000.log
2024-06-26 15:10:19,375 [INFO] executors.py:491 -- ExecutorID 37a208-4 - Getting results from 1 function activations
2024-06-26 15:10:19,376 [INFO] wait.py:101 -- ExecutorID 37a208-4 - Waiting for 1 function activations to complete
2024-06-26 15:11:00,444 [INFO] executors.py:615 -- ExecutorID 37a208-4 - Cleaning temporary data
2024-06-26 15:11:00 [INFO] target Calibration completed in 42.17457437515259 seconds.


In [6]:
TARGET_imaging_params = [
    "-size", "1024", "1024",
    "-pol", "I",
    "-scale", "5arcmin",
    "-niter", "100000",
    "-gain", "0.1",
    "-mgain", "0.6",
    "-auto-mask", "5",
    "-local-rms",
    "-multiscale",
    "-no-update-model-required",
    "-make-psf",
    "-auto-threshold", "3",
    "-weight", "briggs", "0",
    "-data-column", "CORRECTED_DATA",
    "-nmiter", "0",
    "-name", OutputS3(
        bucket=BUCKET,
        key=prepend_hash_to_key("TAR/imag_out/"),
        file_name="image"
    ),
]
# TARGET IMAGING
start_time = time.time()
finished_job = ImagingStep(
    input_data_path=InputS3(
        bucket=BUCKET, key=prepend_hash_to_key("TAR/applycal_out/ms")
    ),
    parameters=TARGET_imaging_params,
    log_level=LOG_LEVEL
).run()
end_time = time.time()
logger.info(f"TARGET Imaging completed in {end_time - start_time} seconds.")

2024-06-26 15:11:00,475 [INFO] config.py:139 -- Lithops v3.4.1 - Python3.10
2024-06-26 15:11:00,494 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:11:00,524 [INFO] k8s.py:111 -- Kubernetes client created - Namespace: default
2024-06-26 15:11:00,536 [INFO] minio.py:62 -- MinIO client created - Endpoint: http://192.168.5.24:9000
2024-06-26 15:11:00,549 [INFO] invokers.py:107 -- ExecutorID 37a208-5 | JobID A000 - Selected Runtime: docker.io/ayman321/extractdysco3 - 8000MB


Initialized OutputS3 with bucket: os-10gb, key: 440531/TAR/imag_out/image, file_ext: None, file_name: image, remote_key_ow: None, base_local_path: /tmp
Initialized InputS3 with bucket: os-10gb, key: 440531/TAR/applycal_out/ms, file_ext: None, dynamic: False, base_local_path: /tmp


2024-06-26 15:11:01,513 [INFO] invokers.py:174 -- ExecutorID 37a208-5 | JobID A000 - Starting function invocation: _execute_step() - Total: 1 activations
2024-06-26 15:11:01,594 [INFO] invokers.py:213 -- ExecutorID 37a208-5 | JobID A000 - View execution logs at /tmp/lithops-ayman/logs/37a208-5-A000.log
2024-06-26 15:11:01 [INFO] parameters: ['-size', '1024', '1024', '-pol', 'I', '-scale', '5arcmin', '-niter', '100000', '-gain', '0.1', '-mgain', '0.6', '-auto-mask', '5', '-local-rms', '-multiscale', '-no-update-model-required', '-make-psf', '-auto-threshold', '3', '-weight', 'briggs', '0', '-data-column', 'CORRECTED_DATA', '-nmiter', '0', '-name', OutputS3(bucket=os-10gb, key=440531/TAR/imag_out/image, file_ext=None, file_name=image, remote_key_ow=None, base_local_path=/tmp), '-j', '10']
2024-06-26 15:11:01,599 [INFO] executors.py:491 -- ExecutorID 37a208-5 - Getting results from 1 function activations
2024-06-26 15:11:01,600 [INFO] wait.py:101 -- ExecutorID 37a208-5 - Waiting for 1 fun